In [22]:
#!pip install auto-sklearn
import pandas as pd
import autosklearn
import autosklearn.regression
from benchmark_utils import timer


In [23]:
X_train = pd.read_csv("../datasets/house-prices/train.csv")
X_test = pd.read_csv("../datasets/house-prices/test.csv")



In [26]:
X_train.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [27]:
X_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [28]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [29]:
## Change target dtype 
y_train = pd.to_numeric(X_train['SalePrice'])
# Drop id and target from train frame
X_train.drop(columns=['SalePrice','Id'],inplace=True)
## Copy test id's and drop id from test frame
id_test = X_test.Id
X_test.drop(columns=['Id'],inplace=True)
#X_train = pd.get_dummies(X_train)
#X_test = pd.get_dummies(X_test)

In [30]:
df_train = pd.get_dummies(X_train, columns=[ column for column in X_train if X_train[column].dtype == "object"])
df_test = pd.get_dummies(X_test, columns=[ column for column in X_train if X_train[column].dtype == "object"])
    

In [33]:
df_test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [8]:

X_train.head()

[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 38, 39, 40, 41, 52, 54, 56, 57, 59, 62, 63, 64, 71, 72, 73, 77, 78]


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 288 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(3), int64(33), uint8(252)
memory usage: 770.0 KB


## Treinamento

In [7]:
start_time = timer(None)
ak = autosklearn.regression.AutoSklearnRegressor(ml_memory_limit=129024)

ak.fit(X_train, y_train)

time = timer(start_time)


/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-11-06 15:33:31,577:AutoMLSMBO(1)::8e012059d89d6724cab4f9507d977436] Could not find meta-data directory /home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/metalearning/files/r2_regression_dense
[WARNING] [2019-11-06 15:33:31,602:EnsembleBuilder(1):8e012059d89d6724cab4f9507d977436] No models better than random - using Dummy Score!
[WARNING] [2019-11-06 15:33:31,629:EnsembleBuilder(1):8e012059d89d6724cab4f9507d977436] No models better than random - using Dummy Score!
[WARNING] [2019-11-06 15:33:33,656:EnsembleBuilder(1):8e012059d89d6724cab4f9507d977436] No models better than random - using Dummy Score!


/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train

/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/midas/anaconda3/envs/automl-3.7/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train

1
['/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000005.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000006.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000007.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000008.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000009.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000010.ensemble', '/tmp/autosklearn_tmp_20857_9944/.auto-sklearn/ensembles/1.0000000011.ensemble', '/tmp/autosklearn_tmp_208

In [10]:
import sklearn
results_df = ak.cv_results_
print(results_df)
preds =  ak.predict(X_test)
print(preds)

{'mean_test_score': array([ 9.09690977e-01,  8.39069247e-01,  4.77413339e-01, -6.80246924e-02,
       -1.55577964e+02,  6.55867620e-01,  0.00000000e+00, -1.16630249e+02,
        6.68560503e-01,  4.60677374e-01,  5.30084462e-01, -5.16282332e-05,
       -1.27464951e+02,  7.48607355e-01,  3.02362462e-01,  8.53242119e-01,
        7.21960497e-01,  9.11342344e-01,  7.21644932e-01, -6.95598095e-01,
        8.66000265e-01,  8.53937851e-01,  8.48281368e-01,  8.19661411e-01,
        8.24777740e-01,  7.21971473e-01,  2.87234540e-05,  7.70954306e-01,
        1.78221996e-01,  7.28480414e-01,  7.97623185e-01, -2.47765650e+01,
       -3.64128487e+00,  8.09964507e-01,  4.51041183e-01,  0.00000000e+00,
        3.10308947e-01, -3.20499113e+00,  8.74041922e-01,  2.65768095e-01,
        5.12947954e-01,  7.76240847e-01,  8.53937851e-01, -2.34056725e+00,
       -1.18146399e+02, -2.22059011e+00,  6.18044261e-01,  8.20443901e-01,
        8.71145846e-01,  7.52003255e-01,  7.66505798e-01, -2.69707355e-04,
     

ValueError: X has 270 features per sample, expected 288

In [ ]:
print(time)

In [ ]:
submission = pd.DataFrame({
        "id": id_test,
        "target": preds
    })

submission.to_csv("../submit_files/house_prices_autosk_submission.csv", index=False)